# LangChain Expression Language (LCEL) Tutorial

In [33]:
from typing import Any, Dict, List

from dotenv import load_dotenv
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser

# Import LangChain components
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import (
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_ollama import ChatOllama
from pydantic import BaseModel
from rich import print

In [34]:
# Load environment variables
load_dotenv()

# Initialize the LLM
llm = ChatOllama(model="llama3.2",temperature=0.3)

## SECTION 1: Basic LCEL Chain

In [35]:
print("-" * 50,'\n',"DEMO 1: Basic LCEL Chain",'\n',"-" * 50)

# Create a simple prompt template
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}.")

# Create a basic chain using the | operator (pipe)
basic_chain = prompt | llm | StrOutputParser()


print("Prompt: ", prompt.format_prompt(topic="cats").to_messages())
print("Chain: ", basic_chain.invoke({"topic": "cats"}))

-------------------------------------------------- 
 DEMO 1: Basic LCEL Chain 
 --------------------------------------------------

Prompt: 
[HumanMessage(content='Tell me a short joke about cats.', additional_kwargs={}, response_metadata={})]

Chain:  Why did the cat join a band?

Because it wanted to be the purr-cussionist!

In [36]:
# Run the chain
result = basic_chain.invoke({"topic": "programming"})
print(result)
print("\n")

Why do programmers prefer dark mode?

Because light attracts bugs.

## SECTION 2: Parallel Processing with LCEL

In [37]:
print("-" * 50,'\n',"DEMO 2: Parallel Processing with LCEL",'\n',"-" * 50)

# Create two different prompts
pros_prompt = ChatPromptTemplate.from_template(
    "List 3 pros of {technology} technology."
)
cons_prompt = ChatPromptTemplate.from_template(
    "List 3 cons of {technology} technology."
)

# Create chains for each prompt
pros_chain = pros_prompt | llm | StrOutputParser()
cons_chain = cons_prompt | llm | StrOutputParser()

# Create a parallel chain
parallel_chain = RunnableParallel(pros=pros_chain, cons=cons_chain)

print("Prompt: ", pros_prompt.format_prompt(technology="AI").to_messages())
print("Chain: ", parallel_chain.invoke({"technology": "AI"}))

-------------------------------------------------- 
 DEMO 2: Parallel Processing with LCEL 
 --------------------------------------------------

Prompt: 
[HumanMessage(content='List 3 pros of AI technology.', additional_kwargs={}, response_metadata={})]

Chain: 
{
    'pros': 'Here are three pros of AI technology:\n\n1. **Improved Efficiency and Productivity**: Artificial 
intelligence can automate repetitive and mundane tasks, freeing up human time and energy for more complex and 
creative work. This can lead to increased productivity, improved accuracy, and enhanced overall efficiency.\n\n2. 
**Enhanced Decision-Making**: AI algorithms can analyze vast amounts of data quickly and accurately, providing 
insights that may not be apparent to humans. This can help organizations make better decisions, identify trends, 
and predict outcomes.\n\n3. **Personalized Experiences and Improved Customer Service**: AI-powered chatbots and 
virtual assistants can provide personalized experiences for customers, responding to their queries and preferences 
in real-time. This can lead to improved customer satisfaction, increased loyalty, and enhanced overall customer 
experience.',
    'cons': 'Here are three cons of AI technology:\n\n1. **Job Displacement**: The increasing use of AI in various 
industries can lead to job displacement, particularly in sectors where tasks are repetitive or can be easily 
automated. This can result in significant unemployment and economic disruption.\n\n2. **Bias and Discrimination**: 
AI systems can perpetuate existing biases and discriminatory practices if they are trained on biased data or 
designed with a particular worldview. This can lead to unfair outcomes in areas such as hiring, law enforcement, 
and healthcare.\n\n3. **Loss of Human Touch and Emotional Intelligence**: Over-reliance on AI can lead to a loss of
human touch and emotional intelligence, particularly in customer service and other fields where empathy and 
understanding are essential. While AI can provide quick answers and solutions, it often lacks the nuance and 
compassion that humans bring to complex problems.'
}

In [38]:

# Run the parallel chain
parallel_result = parallel_chain.invoke({"technology": "blockchain"})
print("PROS:")
print(parallel_result["pros"])
print("\nCONS:")
print(parallel_result["cons"])
print("\n")

PROS:

Here are three pros of blockchain technology:

1. **Security and Transparency**: Blockchain technology uses a decentralized, distributed ledger that records 
transactions across a network of computers. This makes it virtually impossible to alter or manipulate the data, 
ensuring the integrity and security of the information.

2. **Immutable and Tamper-Proof**: The use of cryptography and complex algorithms in blockchain technology ensures 
that once data is recorded on the blockchain, it cannot be altered or deleted. This provides a high level of trust 
and reliability in transactions and data storage.

3. **Decentralized and Autonomous**: Blockchain technology operates independently of central authorities, allowing 
for peer-to-peer transactions and decision-making without the need for intermediaries. This decentralized approach 
enables greater autonomy, efficiency, and scalability in various applications, such as supply chain management and 
voting systems.

CONS:

Here are three cons of blockchain technology:

1. **Scalability limitations**: Blockchain technology is still in its early stages, and many blockchain networks 
face scalability issues due to the limited number of transactions that can be processed per second. This can lead 
to slow transaction processing times, high fees, and congestion on the network.

2. **Energy consumption**: The process of validating transactions on a blockchain requires significant 
computational power, which often relies on energy-intensive mining operations. This has led to concerns about the 
environmental impact of blockchain technology, particularly for proof-of-work (PoW) consensus algorithms like 
Bitcoin.

3. **Regulatory uncertainty and complexity**: Blockchain technology operates in a gray area between traditional 
financial regulations and emerging regulatory frameworks. This can create uncertainty and complexity for businesses
and individuals looking to use blockchain-based solutions, as they may not be aware of the specific laws and 
regulations that apply to their activities.

## SECTION 3: Advanced Chain with Data Transformation

In [40]:
print("-" * 50,'\n',"DEMO 3: Advanced Chain with Data Transformation",'\n',"-" * 50)

# Define a function to process input
def preprocess_input(input_data: Dict[str, Any]) -> Dict[str, Any]:
    """Process the input data before sending to the LLM."""
    # Add a timestamp, capitalize the query, etc.
    processed_data = input_data.copy()
    processed_data["query"] = input_data["query"].upper()
    processed_data["word_count"] = len(input_data["query"].split())
    return processed_data


# Define a function to process output
def postprocess_output(output: str) -> Dict[str, Any]:
    """Process the output from the LLM."""
    lines = output.strip().split("\n")
    return {
        "summary": lines[0] if lines else "",
        "details": lines[1:],
        "response_length": len(output),
    }


# Create an advanced prompt
advanced_prompt = ChatPromptTemplate.from_template(
    """Answer the following query: {query}

    The query has {word_count} words.
    Provide a detailed explanation.
    """
)

# Create an advanced chain with preprocessing and postprocessing
advanced_chain = (
    RunnableLambda(preprocess_input)
    | advanced_prompt
    | llm
    | StrOutputParser()
    | RunnableLambda(postprocess_output)
)

# Preprocess the input data
preprocessed_data = preprocess_input({"query": "Explain quantum computing"})

print("Prompt: ", advanced_prompt.format_prompt(**preprocessed_data).to_messages())
print("Chain: ", advanced_chain.invoke(preprocessed_data))

-------------------------------------------------- 
 DEMO 3: Advanced Chain with Data Transformation 
 --------------------------------------------------

Prompt: 
[
    HumanMessage(
        content='Answer the following query: EXPLAIN QUANTUM COMPUTING\n\n    The query has 3 words.\n    Provide a
detailed explanation.\n    ',
        additional_kwargs={},
        response_metadata={}
    )
]

Chain: 
{
    'summary': "Quantum computing is a revolutionary technology that leverages the principles of quantum mechanics 
to perform calculations and operations on data. It's a field that has gained significant attention in recent years 
due to its potential to solve complex problems that are currently unsolvable or require an unfeasible amount of 
time to solve using classical computers.",
    'details': [
        '',
        '**What is Quantum Computing?**',
        '',
        'Quantum computing is based on the idea that quantum-mechanical phenomena, such as superposition and 
entanglement, can be harnessed to perform calculations. In a classical computer, information is represented as 
bits, which can have a value of either 0 or 1. In contrast, quantum computers use qubits (quantum bits), which can 
exist in multiple states simultaneously, known as superposition.',
        '',
        '**Key Principles of Quantum Computing**',
        '',
        '1. **Superposition**: Qubits can exist in multiple states at the same time, allowing for the simultaneous 
processing of multiple possibilities.',
        '2. **Entanglement**: Qubits can become "entangled," meaning that their properties are connected, even when
separated by large distances.',
        '3. **Quantum Measurement**: When a qubit is measured, its state collapses to one of the possible 
outcomes.',
        '',
        '**How Does Quantum Computing Work?**',
        '',
        '1. **Qubit Creation**: Qubits are created using quantum gates, which are the quantum equivalent of logic 
gates in classical computing.',
        '2. **Quantum Circuit**: A series of quantum gates are applied to qubits to perform a computation.',
        '3. **Quantum Measurement**: The final state of the qubits is measured, collapsing the superposition into 
one of the possible outcomes.',
        '',
        '**Advantages of Quantum Computing**',
        '',
        '1. **Exponential Scaling**: Quantum computers can solve certain problems much faster than classical 
computers, thanks to the exponential scaling of qubits.',
        '2. **Simulations**: Quantum computers can simulate complex systems, such as molecules and chemical 
reactions, which is crucial for fields like chemistry and materials science.',
        '3. **Cryptography**: Quantum computers can break certain types of classical encryption algorithms, but 
they can also be used to create new, quantum-resistant encryption methods.',
        '',
        '**Challenges and Limitations**',
        '',
        '1. **Error Correction**: Quantum computers are prone to errors due to the fragile nature of qubits.',
        '2. **Scalability**: Currently, most quantum computers are small-scale and need to be scaled up to perform 
practical computations.',
        '3. **Quantum Noise**: Quantum computers are susceptible to noise, which can cause errors in 
calculations.',
        '',
        '**Applications of Quantum Computing**',
        '',
        '1. **Cryptography**: Quantum-resistant encryption methods, such as lattice-based cryptography.',
        '2. **Optimization**: Solving complex optimization problems, such as logistics and supply chain 
management.',
        '3. **Materials Science**: Simulating the behavior of materials at the atomic level.',
        '4. **Artificial Intelligence**: Quantum algorithms for machine learning and natural language processing.',
        '',
        '**Current State of Quantum Computing**',
        '',
        'Quantum computing is still in its early stages, with many challenges to overcome before it can be widely 
adopted. However, significant progress has been made in recent years, with companies like Google, IBM, and 
Microsoft investing heavily in quantum computing research and development.',
        '',
        "In summary, quantum computing is a revolutionary technology that leverages the principles of quantum 
mechanics to perform calculations and operations on data. While it's still in its

In [41]:
# Run the advanced chain
advanced_result = advanced_chain.invoke(
    {"query": "how does LCEL improve LLM applications"}
)
print(f"Summary: {advanced_result['summary']}")
print("Details:")
for detail in advanced_result["details"]:
    if detail.strip():
        print(f"- {detail.strip()}")
print(f"Response Length: {advanced_result['response_length']} characters")
print("\n")

Summary: LCE (Loss Constrained Embeddings) is a technique used to improve Large Language Model (LLM) applications. 
Here's a detailed explanation of how LCE improves LLM applications:

Details:

- **What are Loss Constrained Embeddings?**

- Loss Constrained Embeddings is a method for training neural networks, including LLMs, to optimize their 
performance on specific tasks while preventing overfitting. The goal is to find a balance between the model's 
ability to generalize well and its capacity to fit the training data.

- **How does LCE improve LLM applications?**

...

- 1. **Question answering**: LCE can help improve question answering models by reducing overfitting and improving 
generalization.

- 2. **Text classification**: LCE can improve text classification models by reducing dimensionality and improving 
interpretability.

- 3. **Language translation**: LCE can improve language translation models by reducing overfitting and improving 
robustness to noise.

- In summary, LCE improves LLM applications by providing regularization, improving generalization, reducing 
dimensionality, improving interpretability, and increasing robustness to noise. By constraining the embedding 
space, LCE helps prevent overfitting and promotes more generalizable and robust models.

Response Length: 2994 characters

## SECTION 4: RAG Pattern with LCEL

In [ ]:
print("-" * 50,'\n',"DEMO 4: RAG Pattern with LCEL (Simulated)",'\n',"-" * 50)


# Simulate a retriever function
def simulate_retrieval(query: str) -> List[str]:
    """Simulate document retrieval based on query."""
    # In a real application, this would query a vector database
    documents = [
        "LCEL allows for declarative chain construction in LangChain.",
        "LangChain Expression Language simplifies building complex LLM applications.",
        "LCEL supports streaming, async operations, and parallel execution.",
    ]
    return documents


# Create a RAG prompt
rag_prompt = ChatPromptTemplate.from_template(
    """Answer the question based on the following context:

    Context:
    {context}

    Question: {question}
    """
)


# Create a RAG chain
def format_docs(docs):
    return "\n".join(docs)


rag_chain = (
    RunnableParallel(
        {
            "context": RunnableLambda(simulate_retrieval) | RunnableLambda(format_docs),
            "question": RunnablePassthrough(),
        }
    )
    | rag_prompt
    | llm
    | StrOutputParser()
)

print("Prompt: ", rag_prompt.format_prompt(context="context", question="question").to_messages())
print("Chain: ", rag_chain.invoke({"question": "What is LCEL?"}))

In [ ]:
# Run the RAG chain
rag_result = rag_chain.invoke("What is LCEL and why is it useful?")
print(rag_result)